In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
from weaviate.classes.config import Configure
from weaviate.classes.config import Property, DataType, Tokenization

client.collections.create(
    name="<COLLECTION_NAME>",
    # Use text2vec-weaviate vectorizer https://weaviate.io/developers/wcs/embeddings/models#how-to-choose-the-right-model
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-m-v1.5",
        vectorize_collection_name=False
    ), 
    inverted_index_config=Configure.inverted_index(
        index_timestamps = True
    ),
    properties=[
        Property(
            name="<PROPERTY_NAME>",
            data_type=DataType.TEXT,
            tokenization=Tokenization.WORD,
        ),
        Property(
            name="<PROPERTY_NAME>",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD,
        )
    ]
)

In [ ]:
# Insert some data for a collection
data = [
    {
        "Trigger": "I want to buy a new car",
        "Content": "I am looking for a new car, preferably a Tesla."
    },
    {
        "Trigger": "I want to buy a new house",
        "Content": "I am looking for a new house, preferably in the city."
    },
    {
        "Trigger": "I want to buy a new phone",
        "Content": "I am looking for a new phone, preferably an iPhone."
    }
]

coll = client.collections.get("<COLLECTION_NAME>")
coll.data.insert_many(data)